In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import bokeh
import folium
import datetime as dt
%matplotlib inline

In [2]:
# Import libraries
import math
# import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column, layout, Spacer
from bokeh.models.widgets import DateSlider
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange, DateRangeSlider, BoxAnnotation
from bokeh.palettes import GnBu3, OrRd3
from bokeh.plotting import figure
from bokeh.palettes import Spectral6

In [3]:
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange, DateRangeSlider, BoxAnnotation
from bokeh.palettes import GnBu3, OrRd3
from bokeh.plotting import figure
from bokeh.palettes import Spectral6

In [5]:
import pandas as pd
datafile = './data/covid_de.csv'
#Read csv file using pandas
df = pd.read_csv(datafile)
df_demo = pd.read_csv("data/demographics_de.csv")
#Changing gender in demografic to have same values as covid data set. 
df_demo['gender'] = df_demo['gender'].map({'female': 'F','male':'M'})
df = df.dropna()
df['date'] = pd.to_datetime(df.date)
df.date = pd.to_datetime(df["date"], format="%Y-%m-%dT%H:%M:%S.%fZ")
df = df.sort_values(by='date')
df = pd.merge(df,df_demo,how='inner', on=["state","age_group","gender"])
print(df.shape)
df.head()

(47556, 9)


,state,county,age_group,gender,date,cases,deaths,recovered,population
0,Baden-Wuerttemberg,LK Boeblingen,15-34,M,2020-01-05,1,0,1,1423367
1,Baden-Wuerttemberg,LK Ravensburg,15-34,M,2020-02-23,1,0,1,1423367
2,Baden-Wuerttemberg,SK Karlsruhe,15-34,M,2020-02-23,1,0,1,1423367
3,Baden-Wuerttemberg,LK Zollernalbkreis,15-34,M,2020-02-24,1,0,1,1423367
4,Baden-Wuerttemberg,LK Goeppingen,15-34,M,2020-02-25,1,0,1,1423367


#### Location of State
Gotten from: https://www.distancelatlong.com/distancecalculator/country/germany

In [6]:
state_loc = pd.DataFrame(np.array([
['Baden-Wurttemberg',48.99999229,8.999993448],
['Bayern',49.26660748,10.96660681],
['Berlin',52.52181866,13.40154862],
['Brandenburg',51.7704175,13.72996741],
['Bremen',53.05043805,8.779982461],
['Hamburg',53.55002464,9.999999144],
['Hessen',50.30000694,9.200029662],
['Mecklenburg Vorpommern',53.63330408,12.41669861],
['Niedersachsen',52.1299986,9.720004434],
['Nordrhein Westfalen',51.52998822,8.030011351],
['Rheinland Pfalz',49.58247246,7.703219156],
['Saarland',49.25039044,6.970003213],
['Sachsen',50.82998395,12.91997595],
['Sachsen Anhalt',52.13042137,11.62000362],
['Schleswig Holstein',54.28374778,9.433315388],
['Thuringen',50.97005292,11.02996212]]),columns=['State','Long','Lat'])
state_loc = state_loc.sort_values('State')

In [7]:
state_loc['Total Cases'] = df.groupby('state')['cases'].sum().values
state_loc['Total Deaths'] = df.groupby('state')['deaths'].sum().values
state_loc['Total Recovered'] = df.groupby('state')['recovered'].sum().values
state_loc

,State,Long,Lat,Total Cases,Total Deaths,Total Recovered
0,Baden-Wurttemberg,48.99999229,8.999993448,24807,708,12161
1,Bayern,49.26660748,10.96660681,33136,867,15027
2,Berlin,52.52181866,13.40154862,4599,56,3083
3,Brandenburg,51.7704175,13.72996741,1912,40,1195
4,Bremen,53.05043805,8.779982461,484,17,323
5,Hamburg,53.55002464,9.999999144,3778,59,2005
6,Hessen,50.30000694,9.200029662,6107,145,3807
7,Mecklenburg Vorpommern,53.63330408,12.41669861,615,11,435
8,Niedersachsen,52.1299986,9.720004434,7807,210,4776
9,Nordrhein Westfalen,51.52998822,8.030011351,25211,543,16142


In [8]:
mapp = folium.Map(location=[51.4657,10.4515],zoom_start = 5.5, tiles='Stamen Toner')
for i in range(state_loc.shape[0]):
    icon = folium.Icon(color="red")
    folium.Marker([state_loc['Long'][i],state_loc['Lat'][i]],
                  tooltip='State: {} <br>' 'Total Cases: {} <br>' 'Total Deaths: {}<br>' 'Total Recovered: {}'.format(state_loc['State'][i],state_loc['Total Cases'][i],state_loc['Total Deaths'][i],state_loc['Total Recovered'][i]),icon=icon).add_to(mapp)
mapp

## Overlook of Cases, Deaths and Recovered

In [9]:
# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


def niceplot(State,Value):
    
    if Value == 'Total':
        df1 = df[df.state == State]
        df1 = df1.sort_values(by='date')
        df1 = df1[['date','cases','deaths','recovered']]
        df1['Total Cases'] = df1['cases'].cumsum()
        df1['Total Deaths'] = df1['deaths'].cumsum()
        df1['Total Recovered'] = df1['recovered'].cumsum()
        df1 = df1.drop(['cases','deaths','recovered'],axis=1)
        df1 = df1.set_index('date')
        df1.iplot(xTitle="Date",
                    yTitle="Occurrences", title="Corona Development in {}".format(State))
    if Value == 'By day':
        df1 = df[df.state == State]
        df1 = df1.sort_values(by='date')
        df1 = df1[['date','cases','deaths','recovered']]
        df1 = df1.set_index('date')
        df1.columns = ['Cases','Deaths','Recovered']
        df1.iplot(xTitle="Date",
                    yTitle="Occurrences", title="Corona Development in {}".format(State), kind='bar')


interact(niceplot, State=df.state.unique(),Value = ['Total','By day']);

interactive(children=(Dropdown(description='State', options=('Baden-Wuerttemberg', 'Bayern', 'Niedersachsen', …

In [10]:
#Don't run!!!!

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

#Define options
opt = list(df.state.unique())
opt.append('Whole Germany')
opt.insert(0, opt.pop())


def niceplot(State,Value):
    
    if State == 'Whole Germany':
        if Value == 'Total':
            df1 = df
            df1 = df1.sort_values(by='date')
            df1 = df1[['date','cases','deaths','recovered']]
            df1['Total Cases'] = df1['cases'].cumsum()
            df1['Total Deaths'] = df1['deaths'].cumsum()
            df1['Total Recovered'] = df1['recovered'].cumsum()
            df1 = df1.drop(['cases','deaths','recovered'],axis=1)
            df1 = df1.set_index('date')
            df1.iplot(xTitle="Date",
                        yTitle="Occurrences", title="Corona Development in {}".format(State))
    if Value == 'By day':
        df1 = df
        df1 = df1.sort_values(by='date')
        df1 = df1[['date','cases','deaths','recovered']]
        df1 = df1.set_index('date')
        df1.columns = ['Cases','Deaths','Recovered']
        df1.iplot(xTitle="Date",
                    yTitle="Occurrences", title="Corona Development in {}".format(State), kind='bar') 
    else:
        if Value == 'Total':
            df1 = df[df.state == State]
            df1 = df1.sort_values(by='date')
            df1 = df1[['date','cases','deaths','recovered']]
            df1['Total Cases'] = df1['cases'].cumsum()
            df1['Total Deaths'] = df1['deaths'].cumsum()
            df1['Total Recovered'] = df1['recovered'].cumsum()
            df1 = df1.drop(['cases','deaths','recovered'],axis=1)
            df1 = df1.set_index('date')
            df1.iplot(xTitle="Date",
                        yTitle="Occurrences", title="Corona Development in {}".format(State))
        if Value == 'By day':
            df1 = df[df.state == State]
            df1 = df1.sort_values(by='date')
            df1 = df1[['date','cases','deaths','recovered']]
            df1 = df1.set_index('date')
            df1.columns = ['Cases','Deaths','Recovered']
            df1.iplot(xTitle="Date",
                        yTitle="Occurrences", title="Corona Development in {}".format(State), kind='bar')


interact(niceplot, State=opt,Value = ['Total','By day']);

interactive(children=(Dropdown(description='State', options=('Whole Germany', 'Baden-Wuerttemberg', 'Bayern', …

## R0 Plot

In [11]:
popu = df_demo.groupby('state')['population'].sum()
popu.values
df_R0_plot = df.groupby('state')['cases','deaths','recovered'].sum()
df_R0_plot['population'] = popu.values

C:\Users\victo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [12]:
from scipy.integrate import odeint
def deriv(y, t, N, beta, gamma, delta, alpha, rho):
    S, E, I, R, D = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - (1 - alpha) * gamma * I - alpha * rho * I
    dRdt = (1 - alpha) * gamma * I
    dDdt = alpha * rho * I
    return dSdt, dEdt, dIdt, dRdt, dDdt

In [21]:
from ipywidgets import IntSlider, FloatSlider, interactive

def R0_plot(State,R0,Days_Ahead):
    datdat = df_R0_plot[df_R0_plot.index == State]

    cumCase = datdat.cases.values.item(0)
    cumDeath = datdat.deaths.values.item(0)
    cumRecovered = datdat.recovered.values.item(0)
    cumExposed = 1 #Exposed (Find ud af hvad der skal stå her)


    N = datdat['population'].values.item(0) #Population
    D = 14 # infections lasts two weeks
    gamma = 1 / D
    delta = 1 / 14  # incubation period of two weeks
    R_0 = R0
    beta = R_0 * gamma  # R_0 = beta / gamma, so beta = R_0 * gamma
    alpha = cumDeath/cumCase  #Calculated death rate
    rho = 1/7  # 7 days from infection until death (Based on https://www.worldometers.info/coronavirus/coronavirus-death-rate/)
    E0 = cumExposed
    S0 = N - cumCase - cumDeath - cumRecovered - E0 #Suspectible
    I0 = cumCase - cumDeath - cumRecovered
    R0 = cumRecovered
    D0 = cumDeath
    
    t = np.linspace(0, Days_Ahead,Days_Ahead) # Grid of time points (in days)
    y0 = S0, E0, I0, R0, D0 # Initial conditions vector

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma, delta, alpha, rho))
    S, E, I, R, D = ret.T
    
    date_list = [df.date.max() + dt.timedelta(days=x) for x in range(Days_Ahead)]
    d = {'Date': date_list, 'Suspectible': S, 'Infected': I, 'Recovered': R, 'Deaths': D}
    plotdat = pd.DataFrame(data=d)
    plotdat = plotdat.set_index('Date')
    fig = plotdat.iplot(xTitle="Date", yTitle="Occurrences",
                        title="SEIR Model in {} <br>" "R0: {:.2f}<br>" "Total deaths: {:,.0f}<br>" "Max infected at once: {:,.0f}".format(State,R_0,max(D),max(I)))
    
interactive(R0_plot, State=df.state.unique(),R0 = FloatSlider(min=0.2,max=10,step=0.1),
                Days_Ahead = IntSlider(min=100, max=1000, step=10),continuous_update=False)

interactive(children=(Dropdown(description='State', options=('Baden-Wuerttemberg', 'Bayern', 'Niedersachsen', …